# Práctica Guiada: Datasets de tablas múltiples- Archivo ENRON

* En esta actividad se explorará un subconjunto de datos de la Base de Datos de E-mails de Enron.

* Este dataset fue preparado y recopilado por el Proyecto CALO (A Cognitive Assistant that Learns and Organizes).

* El dataset original contiene datos de unos 150 usuarios, la mayoría gerentes de Enron y aproximadamente 0,5 millones de mensajes.

* Estos datos fueron publicados en la web por la Comisión Regulatoria de Energía Federal (EEUU) durante su investigación.

## 1. Importar datos

Conectarse al archivo 'enron.db' utilizando alguno de los siguientes métidos:

- Paquete sqlite3 de Python
- pandas.read_sql
- SQLite Manager Firefox extension, DB Browser for SQLite, etc.

Observar la base de datos y consultar la tabla maestra. ¿Cuántas tablas hay en la Base de Datos?

> Respuesta:
Hay 3 tablas:
- MessageBase
- RecipientBase
- EmployeeBase

Recordá intalar bkcharts con: 
> `conda install -c anaconda bkcharts`

In [1]:
import sqlite3
import pandas as pd

con = sqlite3.connect('enron.db')

data = pd.read_sql_query("SELECT * FROM sqlite_master WHERE type='table'", con)
data

,type,name,tbl_name,rootpage,sql
0,table,MessageBase,MessageBase,3,"CREATE TABLE MessageBase (\n mid INTEGER,\n..."
1,table,RecipientBase,RecipientBase,5,"CREATE TABLE RecipientBase (\n mid INTEGER,..."
2,table,EmployeeBase,EmployeeBase,2,CREATE TABLE EmployeeBase (\n ...


Consultar la tabla `sqlite_master` para obtener el schema de la tabla `EmployeeBase`.

1. ¿Cuántos campos tiene?
2. ¿Qué tipo de datos tiene cada uno?

In [3]:
import sqlite3
import pandas as pd

con = sqlite3.connect('enron.db')
for row in con.execute("pragma table_info('EmployeeBase')").fetchall():
    print(row)

(0, 'eid', 'INTEGER', 0, None, 0)
(1, 'name', 'TEXT', 0, None, 0)
(2, 'department', 'TEXT', 0, None, 0)
(3, 'longdepartment', 'TEXT', 0, None, 0)
(4, 'title', 'TEXT', 0, None, 0)
(5, 'gender', 'TEXT', 0, None, 0)
(6, 'seniority', 'TEXT', 0, None, 0)


1. Imprimir las primeras 5 filas de la tabla EmployeeBase
2. Imprimir las primeras 5 filas de la tabla MessageBase
3. Imprimir las primeras 5 filas de la tabla RecipientBase

**Ayuda**  combinar `SELECT` con `LIMIT`.



Importar cada una de las 3 tablas a una estructura Pandas Dataframes

In [4]:
import sqlite3
import pandas as pd

con = sqlite3.connect('enron.db')

data1 = pd.read_sql_query("SELECT * FROM EmployeeBase LIMIT 5", con)
data1

data2 = pd.read_sql_query("SELECT * FROM MessageBase LIMIT 5", con)
data2

data3 = pd.read_sql_query("SELECT * FROM RecipientBase LIMIT 5", con)
data3

,mid,rno,to_eid
0,1,1,59
1,2,1,15
2,3,1,15
3,4,1,109
4,4,2,49


## 2. Exploración de Datos

Utilizar los 3 dataframes para responder las siguientes preguntas:

1. ¿Cuántos empleados hay en la compañía?
2. ¿Cuántos mensajes hay en la Base de Datos?
3. Convertir la columna timestamp en los mensajes. ¿Cuándo fue enviado el mensaje más antiguo? ¿Y el más reciente?
4. Algunos mensajes se enviaron a más de un destinatario. Agrupar los mensajes por message_id y contar la cantidad de destinatarios. Luego, observar la distribución de la cantidad de destinatarios.
    - ¿Cuántos mensajes tienen sólo 1 destinatario?
    - ¿Cuántos mensajes tienen al menos 5 destinatarios?
    - ¿Cuál es la máxima cantidad de destinatarios de un mensaje?
    - ¿Quién envió el mensaje con la máxima cantidad de destinatarios?
5. Imprimir la distribución de la cantidad de destinatarios utilizando Bokeh.

In [6]:
import sqlite3
import pandas as pd
import datetime
import numpy as np
from bokeh.plotting import figure, output_file, show
from bkcharts import Bar, output_file, show
from bokeh.layouts import row

con = sqlite3.connect('enron.db')

data1 = pd.read_sql_query("SELECT * FROM EmployeeBase", con)
data2 = pd.read_sql_query("SELECT * FROM MessageBase", con)
data3 = pd.read_sql_query("SELECT * FROM RecipientBase", con)

# 1
data1['eid'].count() ## 156 empleados

# 2
data2['mid'].count() ## 21635 mensajes

# 3
print(
    datetime.datetime.fromtimestamp(
        int(data2['unix_time'].max())
    ).strftime('%Y-%m-%d %H:%M:%S')
) ## El mas reciente: 2002-06-21 10:37:34

print(
    datetime.datetime.fromtimestamp(
        int(data2['unix_time'].min())
    ).strftime('%Y-%m-%d %H:%M:%S')
) ## El mas antiguo: 1998-11-13 01:07:00

# 4
data3.groupby('mid').size()
## Alternativa usando count() y especificando la columna a contar:
data3.groupby('mid')['rno'].count()

# 4.a
destinatarios = data3.groupby('mid').size()
destinatarios[destinatarios == 1]

# 4.b
destinatarios[destinatarios >= 5]

# 4.c
destinatarios[destinatarios == destinatarios.max()] ## mid = 12116 y 12151 con 57 destinatarios

# 4.d
data2[(data2['mid'] == 12116) | (data2['mid'] == 12151)] ## from_eid = 67
# o bien:
data2[(data2['mid'] == 12116) | (data2['mid'] == 12151)]['from_eid'] ## = 67
# luego buscar el Employee = 67
data1[data1['eid'] == 67] # John J. Lavorato
# Alternativa con MERGE (simil Join) y mostrando los campos mid y name
pd.merge(data1, data2[(data2['mid'] == 12116) | (data2['mid'] == 12151)], left_on='eid', right_on='from_eid')[['mid','name']]

# 5
output_file("distrib_cant_dest.html")

bar = Bar(data3.groupby(data3.groupby(['mid']).size()).size(), title = "Distribucion de la Cantidad de Destinatarios", xlabel='Cant. Destinatarios', ylabel='Cant. Mensajes', legend=None)

show(row(bar))

# Otra forma de resolverlo, definiendo x e y en grafico de linea
# x = data3.groupby(data3.groupby(['mid']).size()).size().keys()
# y = data3.groupby(data3.groupby(['mid']).size()).size().values.tolist()

# p = figure(title="Distribucion de la Cantidad de Destinatarios", x_axis_label='destinatarios', y_axis_label='cant. mensajes')

# p.line(x, y, legend="Cant.", line_width=2)
# show(p)


2002-06-21 10:37:34
1998-11-13 01:07:00


Cambiar la escala para investigar la cola de la curva (OPCIONAL).

## 3. Combinación de datos

Utilizar la función `merge` de Pandas para combinar la información de los 3 dataframes y responder a las siguientes preguntas:

1. ¿Hay más empleados Hombres o Mujeres?
2. ¿Cómo se distribuye el género a través de los departamentos?
3. ¿Quiénes envían más e-mails, los hombres o las mujeres?
4. ¿Cuál es el promedio de e-mails enviados por cada género?
5. ¿Hay más Juniors o Seniors?
6. ¿Quiénes envían más e-mails, los Juniors o los Seniors?
7. ¿Qué departamento envía más e-mails? ¿Cómo se relaciona con la cantidad de empleados del departamento?
8. ¿Cuál es el Top 3 de enviadores de e-mails? (los 3 empleados que más e-mails envían)

In [9]:
import sqlite3
import pandas as pd
import datetime
import numpy as np

con = sqlite3.connect('enron.db')

data1 = pd.read_sql_query("SELECT * FROM EmployeeBase", con)
data2 = pd.read_sql_query("SELECT * FROM MessageBase", con)
data3 = pd.read_sql_query("SELECT * FROM RecipientBase", con)

print("1.")
if data1.groupby('gender').size()['Male']>data1.groupby('gender').size()['Female']:
    print("Hay más empleados Hombres:", data1.groupby('gender').size()['Male'])
else:
    print("Hay más empleados Mujeres (o igual que Hombres):", data1.groupby('gender').size()['Female'])

print("2.")
print(data1.groupby(['gender','department']).size())

print("3.")
if pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('gender').size()['Male']>pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('gender').size()['Female']:
    print("Los Hombres envían más e-mails:",pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('gender').size()['Male'])
else:
    print("Las Mujeres envían más e-mails:",pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('gender').size()['Female'])

print("4.")
print("En promedio las Mujeres envían",(pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('gender').size()['Female'] / data1.groupby('gender').size()['Female']),"e-mails")
print("En promedio los Hombres envían",(pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('gender').size()['Male'] / data1.groupby('gender').size()['Male']),"e-mails")

print("5.")
if data1.groupby('seniority').size()['Senior']>data1.groupby('seniority').size()['Junior']:
    print("Hay más empleados Senior:", data1.groupby('seniority').size()['Senior'])
else:
    print("Hay más empleados Junior (o igual que Senior):", data1.groupby('seniority').size()['Junior'])
    
print("6.")
if pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('seniority').size()['Senior']>pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('seniority').size()['Junior']:
    print("Los Senior envían más e-mails:",pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('seniority').size()['Senior'])
else:
    print("Los Junior envían más e-mails:",pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('seniority').size()['Junior'])

print("7.")
print(pd.merge(data1, data2, left_on='eid', right_on='from_eid').groupby('department').size())
print(data1.groupby('department').size())

print("8.")
print(data2.groupby('from_eid').size().nlargest(3, "first"))

1.
Hay más empleados Hombres: 113
2.
gender  department
Female  Forestry      10
        Legal         13
        Other         20
Male    Forestry      50
        Legal         12
        Other         51
dtype: int64
3.
Los Hombres envían más e-mails: 12841
4.
En promedio las Mujeres envían 204.511627907 e-mails
En promedio los Hombres envían 113.637168142 e-mails
5.
Hay más empleados Junior (o igual que Senior): 82
6.
Los Senior envían más e-mails: 12439
7.
department
Forestry     4387
Legal       10396
Other        6852
dtype: int64
department
Forestry    60
Legal       25
Other       71
dtype: int64
8.
from_eid
20     1597
59     1379
120    1142
dtype: int64


Responder a las siguientes preguntas de aceurdo a los mensajes recibidos:

1. ¿Quiénes reciben más e-mails, los Hombres o las Mujeres?
2. ¿Quiénes reciben más e-mails, los Juniors o los Seniors?
3. ¿Qué departamento recibe más e-mails? ¿Cómo se relaciona con la cantidad de empleados del departamento?
4. ¿Cuál es el Top 5 de receptores de e-mails? (los 5 empleados que más e-mails recibieron)

In [10]:
import sqlite3
import pandas as pd
import datetime
import numpy as np

con = sqlite3.connect('enron.db')

data1 = pd.read_sql_query("SELECT * FROM EmployeeBase", con)
data2 = pd.read_sql_query("SELECT * FROM MessageBase", con)
data3 = pd.read_sql_query("SELECT * FROM RecipientBase", con)

Hombres_reciben = pd.merge(data1, data3, left_on='eid', right_on='to_eid').groupby('gender').size()['Male']
Mujeres_reciben = pd.merge(data1, data3, left_on='eid', right_on='to_eid').groupby('gender').size()['Female']

Junior_reciben = pd.merge(data1, data3, left_on='eid', right_on='to_eid').groupby('seniority').size()['Junior']
Senior_reciben = pd.merge(data1, data3, left_on='eid', right_on='to_eid').groupby('seniority').size()['Senior']

print("1.")
if Hombres_reciben>Mujeres_reciben:
    print("Los Hombres reciben más e-mails")
else:
    print("Las Mujeres reciben más e-mails (o igual)")

print ("2.")
if Junior_reciben>Senior_reciben:
    print("Los Junior reciben más e-mails")
else:
    print("Las Senior reciben más e-mails (o igual)")

print("3.")
print(pd.merge(data1, data3, left_on='eid', right_on='to_eid').groupby('department').size())
print(data1.groupby('department').size())

print("4.")
print(data3.groupby('to_eid').size().nlargest(5, "first"))


1.
Los Hombres reciben más e-mails
2.
Las Senior reciben más e-mails (o igual)
3.
department
Forestry     8424
Legal       16311
Other       13653
dtype: int64
department
Forestry    60
Legal       25
Other       71
dtype: int64
4.
to_eid
131    1797
122    1730
138    1477
61     1290
120    1173
dtype: int64


¿Cuáles empleados enviaron más e-mails de forma "masiva"?

In [11]:
import sqlite3
import pandas as pd
import datetime
import numpy as np

con = sqlite3.connect('enron.db')

data1 = pd.read_sql_query("SELECT * FROM EmployeeBase", con)
data2 = pd.read_sql_query("SELECT * FROM MessageBase", con)
data3 = pd.read_sql_query("SELECT * FROM RecipientBase", con)

enviadores = pd.merge(data2, data3, on='mid').groupby(['from_eid','mid']).size()
print(enviadores[enviadores > 50]) ## Los eid 67 y 68 son los que más "spam" hicieron




from_eid  mid  
67        12116    57
          12140    55
          12151    57
68        14404    52
dtype: int64


Continuar explorando el dataset. ¿Qué otras preguntas se podrían hacer?

Trabajar en grupos de a 2. Intercambiando preguntas desafiantes para que el otro las responda.